Building a Chatbot with history

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI_KEY")
#Langchain Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_KEY")

In [3]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001AF849B0440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AF849B1010>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
response=llm.invoke([HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai")])
response.content

"Hey Sachin,\n\nThat's awesome! It's great to meet another aspiring AI enthusiast. \n\nThapar University's CSE program is known to be quite strong, so you're in a good spot to learn and grow.  \n\nPracticing Generative AI is really exciting - what specific areas within Generative AI are you focusing on? Are you working on text generation, image synthesis, or something else entirely? \n\nI'm always eager to learn about what others are working on. Maybe we can even bounce some ideas off each other sometime!\n"

In [5]:
from langchain_core.messages import AIMessage
llm.invoke(
    [
        HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai.what will you suggest me for a better learning?") ,
        AIMessage(content="Hi Sachin,\n\nIt's great to meet you!  It's impressive that you're already practicing Generative AI during your B.Tech in CSE at Thapar University.  That's a very forward-looking skillset to be developing.  What specific areas of Generative AI are you focusing on right now?  I'd be interested to hear more about your projects.") ,
        HumanMessage(content="What is my name?")
    ]
)


AIMessage(content='Your name is Sachin Goyal. 😊  \n\nI remembered that from our first interaction! \n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 152, 'total_tokens': 175, 'completion_time': 0.041818182, 'prompt_time': 0.01006478, 'queue_time': 0.24592119899999998, 'total_time': 0.051882962}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--008607b4-68f7-49c3-aeec-38bec7000166-0', usage_metadata={'input_tokens': 152, 'output_tokens': 23, 'total_tokens': 175})

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={} #storing all the chat with their id to uniquely define and prevent chat mixing
def get_session_history(session_id)->BaseChatMessageHistory:
    if session_id not in store:  #if new chat then store the chat going on
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(llm,get_session_history)

In [7]:
#declaring session id chat1 for this chat for a user
config={"configurable":{"session_id":"chat1"}}

response=with_message_history.invoke (
    [ HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai.what will you suggest me for a better learning?") ]
    ,config=config )
response

AIMessage(content="That's awesome, Sachin! It's great to hear you're diving into the exciting world of Generative AI during your B.Tech.  \n\nHere are some suggestions to level up your learning:\n\n**Deepen Your Foundations:**\n\n* **Machine Learning Fundamentals:**  Ensure you have a solid grasp of core ML concepts like supervised/unsupervised learning, deep learning architectures (CNNs, RNNs, Transformers), and common algorithms. There are fantastic online courses on platforms like Coursera, edX, and fast.ai.\n* **Python Proficiency:** Python is the go-to language for AI.  Master its libraries like NumPy, Pandas, Matplotlib, and TensorFlow/PyTorch.  \n\n**Dive into Generative AI Specifically:**\n\n* **Text Generation:** Explore models like GPT-3, BERT, T5, and their open-source counterparts (e.g., GPT-Neo, Bloom). Understand how they work and experiment with fine-tuning them for specific tasks like story writing, dialogue generation, or code completion.\n* **Image Generation:**  Fami

In [8]:
#see this time it says i don't know about you name
#basically the session id is a new one so no previous chat available it will be stored for the first time

config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],config=config
)
response

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to use it!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 13, 'total_tokens': 51, 'completion_time': 0.069090909, 'prompt_time': 0.00206003, 'queue_time': 0.241857872, 'total_time': 0.071150939}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--253518a3-6885-4b6a-b1e7-b95c4cf22a0a-0', usage_metadata={'input_tokens': 13, 'output_tokens': 38, 'total_tokens': 51})

In [9]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="ok tell me what is my name")],config=config
)
response
#see it remembers my name

AIMessage(content="Your name is Sachin Goyal. 😊  I remember it! \n\n\nLet me know if you have any other questions about generative AI or anything else.  I'm here to help!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 583, 'total_tokens': 625, 'completion_time': 0.076363636, 'prompt_time': 0.02583528, 'queue_time': 0.24705448100000002, 'total_time': 0.102198916}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b6b1c79d-2283-42fe-846e-f8c63556c4c0-0', usage_metadata={'input_tokens': 583, 'output_tokens': 42, 'total_tokens': 625})

In [10]:
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke([HumanMessage(content="My name is John")],config=config)
response

AIMessage(content="Hi John, it's nice to meet you!\n\nIs there anything I can help you with today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 63, 'total_tokens': 88, 'completion_time': 0.045454545, 'prompt_time': 0.00379868, 'queue_time': 0.243285469, 'total_time': 0.049253225}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--386b0b2d-c7f8-4ea2-823d-083311b60f87-0', usage_metadata={'input_tokens': 63, 'output_tokens': 25, 'total_tokens': 88})

In [11]:
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],config=config
)
response
#this time remembers the name

AIMessage(content='You said your name is John earlier.  \n\nWould you like me to call you something else? 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 100, 'total_tokens': 125, 'completion_time': 0.045454545, 'prompt_time': 0.005276419, 'queue_time': 0.241617902, 'total_time': 0.050730964}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--3eabfd98-1bc6-484e-9a54-9b4f807008b1-0', usage_metadata={'input_tokens': 100, 'output_tokens': 25, 'total_tokens': 125})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [12]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
     [
         ("system","You are a helpful assistant ,communicate with the user according"),
         MessagesPlaceholder(variable_name="messages")
     ]
 )
chain=prompt|llm

In [13]:
chain.invoke({"messages":[HumanMessage(content="Hii ,My name is Sachin Goyal")]})

AIMessage(content="Hi Sachin Goyal, it's nice to meet you! 👋\n\nHow can I help you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 28, 'total_tokens': 55, 'completion_time': 0.049090909, 'prompt_time': 0.00232549, 'queue_time': 0.2408127, 'total_time': 0.051416399}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--65d3425f-f5d8-43e9-901f-f453ad7c7c91-0', usage_metadata={'input_tokens': 28, 'output_tokens': 27, 'total_tokens': 55})

In [14]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [15]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {'message':[HumanMessage(content="My name is Sachin Goyal")]},config=config
)
response

AIMessage(content="You're right!  I apologize for the mistake.  I'm still under development and learning to remember things accurately.  \n\nThanks for correcting me, Sachin.  \n\nHow can I help you further?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 650, 'total_tokens': 697, 'completion_time': 0.085454545, 'prompt_time': 0.02852218, 'queue_time': 0.24637427, 'total_time': 0.113976725}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f4471881-9184-4ceb-b9a9-45099931d927-0', usage_metadata={'input_tokens': 650, 'output_tokens': 47, 'total_tokens': 697})

In [16]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {'message':[HumanMessage(content="can you tell me what is my age and my fav hobby")]},config=config
)
response

AIMessage(content="As an AI, I don't have access to personal information about you like your age or favorite hobby. \n\nTo help me learn more about you, would you like to tell me your age and favorite hobby? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 717, 'total_tokens': 767, 'completion_time': 0.090909091, 'prompt_time': 0.030922791, 'queue_time': 0.24485893799999997, 'total_time': 0.121831882}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--af9750de-079c-4cad-96cf-9de12508e4e6-0', usage_metadata={'input_tokens': 717, 'output_tokens': 50, 'total_tokens': 767})

What if we have multiple inputs to be taken in prompt
--> then there is a change in with_message_history
    we need to define a correct key to use to save chat to history

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
     [
         ("system","You are a helpful assistant ,communicate with the user according in {language}"),
         MessagesPlaceholder(variable_name="messages")
     ]
 )
chain=prompt|llm

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hii ,My name is Sachin Goyal")],"language":"Hindi"})

AIMessage(content='नमस्ते सचिन गोयल जी! \n\nमुझे बहुत खुशी है आपसे मिलने की। आप मुझसे क्या पूछना चाहेंगे? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 30, 'total_tokens': 72, 'completion_time': 0.076363636, 'prompt_time': 0.00212683, 'queue_time': 0.244149009, 'total_time': 0.078490466}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--8e58cce0-3c7b-429e-bcbd-de2bc7a6f635-0', usage_metadata={'input_tokens': 30, 'output_tokens': 42, 'total_tokens': 72})

In [19]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

whenever we reuse the older session for ex chat1 then it loads the old history and as we used english by default so now declaring any other language may not work for that and will print in english

In [20]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="can you tell me my name ")],"language":"Hindi"},config=config
)
response

AIMessage(content='Your name is Sachin Goyal.  😊  Is there anything else I can help you with today? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 783, 'total_tokens': 808, 'completion_time': 0.045454545, 'prompt_time': 0.02787966, 'queue_time': 0.247722709, 'total_time': 0.073334205}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d1e91ff6-e092-4b42-aad2-d50b5f1f4a10-0', usage_metadata={'input_tokens': 783, 'output_tokens': 25, 'total_tokens': 808})

In [21]:
config={"configurable":{"session_id":"chat5"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is Sachin Goyal")],"language":"Hindi"},config=config
)
response

AIMessage(content='नमस्ते Sachin Goyal जी! 👋\n\nमैं आपका मदद करने के लिए यहाँ हूँ। क्या मैं आपकी कोई मदद कर सकता हूँ? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 28, 'total_tokens': 68, 'completion_time': 0.072727273, 'prompt_time': 0.0021672, 'queue_time': 0.2424125, 'total_time': 0.074894473}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--71f108bc-4a6e-411e-b82a-37eba25c3d6d-0', usage_metadata={'input_tokens': 28, 'output_tokens': 40, 'total_tokens': 68})

In [22]:
config={"configurable":{"session_id":"chat5"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="can you tell me my name ")],"language":"Hindi"},config=config
)
response

AIMessage(content='आपका नाम Sachin Goyal है।  😄 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 83, 'total_tokens': 98, 'completion_time': 0.027272727, 'prompt_time': 0.005552549, 'queue_time': 0.24357124100000002, 'total_time': 0.032825276}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--16ee8e85-2892-4af6-ad6e-45c79b764b8d-0', usage_metadata={'input_tokens': 83, 'output_tokens': 15, 'total_tokens': 98})

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [24]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy='last', # keep the msgs from last with tokens<=max_tokens
    token_counter=llm,
    allow_partial=False, #If a msg pushes total tokens over 45, it won’t partially include it instead, it will stop adding messages.
    include_system=True, # as system guides the model what should it do so it must be present anyhow even after trimming msgs
    start_on='human'  #look for the most recent HumanMessage and start including from there.
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

D:\GEN-AI\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sachin Goyal\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [35]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
        RunnablePassthrough.assign(messages=itemgetter("messages") |trimmer)
       | prompt
       | llm
)
response=chain.invoke(
    {"messages":messages+[HumanMessage(content="what ice cream do i like")],"language":"Hindi"}
)
response

AIMessage(content="मुझे नहीं पता कि आपको कौन सा आइसक्रीम पसंद है! मुझे बताओ, आपका पसंदीदा आइसक्रीम क्या है?  🍦😊 \n\n\n(I don't know what your favorite ice cream is! Tell me, what's your favorite ice cream? 🍦😊) \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 81, 'total_tokens': 153, 'completion_time': 0.130909091, 'prompt_time': 0.00413661, 'queue_time': 0.240947836, 'total_time': 0.135045701}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--9c5f8354-09ae-481a-897e-719f0390c667-0', usage_metadata={'input_tokens': 81, 'output_tokens': 72, 'total_tokens': 153})

In [36]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
        RunnablePassthrough.assign(messages=itemgetter("messages") |trimmer)
       | prompt
       | llm
)
response=chain.invoke(
    {"messages":messages+[HumanMessage(content="what was my question regarding maths")],"language":"Hindi"}
)
response

AIMessage(content='आपने मुझसे 2 + 2 का जवाब पूछा था।  \n\n\n\n(You asked me the answer to 2 + 2.)\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 81, 'total_tokens': 119, 'completion_time': 0.069090909, 'prompt_time': 0.00505106, 'queue_time': 0.24509043000000003, 'total_time': 0.074141969}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--47ccd9f4-abcc-4cfd-b3c6-5e4eff3a88ea-0', usage_metadata={'input_tokens': 81, 'output_tokens': 38, 'total_tokens': 119})

Wrapping with Message history

In [37]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [39]:
config={"configurable":{"session_id":"chat6"}}
response=with_message_history.invoke(
    {"messages":messages+[HumanMessage(content="what is my name")],"language":"Hindi"},config=config
)
response

AIMessage(content="मैं आपके नाम को नहीं जानता। आपका नाम क्या है?  \n \n(I don't know your name. What's your name?)\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 79, 'total_tokens': 116, 'completion_time': 0.067272727, 'prompt_time': 0.00417959, 'queue_time': 0.24175738, 'total_time': 0.071452317}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ece0cbdc-40ea-4e4b-a5c9-6ba4f56a5536-0', usage_metadata={'input_tokens': 79, 'output_tokens': 37, 'total_tokens': 116})

In [42]:
config={"configurable":{"session_id":"chat6"}}
response=with_message_history.invoke(
    {"messages":messages+[HumanMessage(content="My name is Sachin,can you tell me the meaning of this ")],"language":"Hindi"},config=config
)
response

AIMessage(content='Sachin नाम का मतलब है "सफलता" या "विजय"। यह एक बहुत ही सुंदर और प्रचलित नाम है। 😊\n\nक्या आप कुछ और जानना चाहेंगे?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 72, 'total_tokens': 118, 'completion_time': 0.083636364, 'prompt_time': 0.003864509, 'queue_time': 0.24411575100000002, 'total_time': 0.087500873}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--15778390-bb81-4f22-a88c-64caee2f5a48-0', usage_metadata={'input_tokens': 72, 'output_tokens': 46, 'total_tokens': 118})